In [52]:
# Import all necessary packages

import json
import time

from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [51]:
api_dict = {}
version = ""

try:
    # Use Firefox as the browser.
    # To use in linux, download geckodriver for firefox
    # https://sc.vern.cc/@seehleung/setup-geckodriver-and-chromedriver-for-selenium-in-linux-2e3f5f462c79
    driver = webdriver.Firefox()
    driver.get('https://developer.gst.gov.in/apiportal/taxpayer/returns/')

    # Wait till browser opens, page loads and the content populated
    time.sleep(10)

    # Click on 'View API List'
    element = driver.find_element(By.XPATH, "//a[@class='inverseLink id1' and @ng-click=\"apilist('returns','Returns')\"]")
    element.click()

    # Wait for page to load
    time.sleep(5)

    # Loop over all 27 pages
    # TODO: Fetch page numbers from html
    #       Bit lazy to do it now
    for i in range(4):

        # Get all API items. For some reason, they are all inside an unordered tag
        ul_elements = driver.find_elements(By.CSS_SELECTOR, "ul.mar-t-10")
        for ul_element in ul_elements:
            # Click on the API link, this will open the api details
            anchor_tag = ul_element.find_element(By.TAG_NAME, "a")
            anchor_tag.click()
            time.sleep(3)

            api_dict[anchor_tag.text] = {}

            # Get API description
            desc = driver.find_element(By.CLASS_NAME, "text-justify")
            api_dict[anchor_tag.text]["description"] = desc.text

            # Populate api with its version details
            tbody = driver.find_element(By.TAG_NAME, "tbody")
            rows = tbody.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                for idx, cell in enumerate(cells):
                    if idx == 0:
                        version = cell.text
                        api_dict[anchor_tag.text]["apis"] = {}
                        api_dict[anchor_tag.text]["apis"][version] = []
                    else:
                        api_dict[anchor_tag.text]["apis"][version].append(cell.text)

        # Click on the next page
        element = driver.find_element(By.XPATH, "/html/body/div[1]/div/ng-view/div/div/div[3]/div/div/div/ul/li[11]/a")
        element.click()

        # Wait till the page loads
        time.sleep(5)
finally:
    # Close the WebDriver
    driver.quit()

{'All - Download Document': {'description': 'This API is used to Download Document', 'apis': {'v1.0': ['30-05-2018', '', 'Draft']}}, 'All - Get additional late fee breakup': {'description': 'This API will help taxpayer to fetch the break up of the additonal late fee posted in their current return period', 'apis': {'v1.1': ['08-03-2021', '', 'Mock']}}, 'All - Get File Details': {'description': 'This API is called with the file_num to get the file URL', 'apis': {'v0.3': ['09-06-2017', '', 'Draft']}}, 'All - GET PREFERENCE': {'description': 'API call to get preference for taxpayers', 'apis': {'v1.0': ['31-08-2020', '', 'Draft']}}, 'All - Get Return Status': {'description': 'This API is to get the status of the Returns submitted', 'apis': {'v0.3': ['30-06-2017', '', 'Draft']}}, 'All - Get User Masters': {'description': 'This API will be used by G2B to Get the User Masters - Products and Supplier Recipients', 'apis': {'v1.0': ['01-01-2021', '', 'Draft']}}, 'All - GetDocStatus': {'descriptio

In [54]:
with open("apis.json", "w") as f:
    json.dump(api_dict, f)